In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/andy/Documents/Research/software/basecode/')
from importscript import *

no_warnings()

In [2]:
import scipy

In [3]:
def get_params(gmm_inst, Nstar, n_components):
    
    params = np.zeros((n_components,6))
    params[:,:2] = gmm_inst.means_
    params[:,2:4] = gmm_inst.covariances_[:,[0,1],[0,1]]
    params[:,4] = gmm_inst.covariances_[:,0,1]/np.sqrt(gmm_inst.covariances_[:,0,0]*gmm_inst.covariances_[:,1,1])
    params[:,5] = gmm_inst.weights_*Nstar

    return params

In [77]:
params = np.array([[1., 1., 2., 2., 0.1, 10.],
                   [-1., -1., 2., 2., 0., 5.],
                   [-0.1, -0.1, 2., 2., 0., 3.]])

In [8]:
params[:,0]

array([ 1., -1.])

In [24]:
def quick_invdet(S):
    det = S[:,0,0]*S[:,1,1] - S[:,0,1]**2
    Sinv = S.copy()*0
    Sinv[:,0,0] = S[:,1,1]
    Sinv[:,1,1] = S[:,0,0]
    Sinv[:,0,1] = -S[:,0,1]
    Sinv[:,1,0] = -S[:,1,0]
    Sinv *= 1/det[:,np.newaxis,np.newaxis]
    
    return Sinv, det

In [44]:
def rootfinder(X, sigma_inv, sigma_det, mu, weight, norm):
    
    delta = mu-X[:,np.newaxis]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = np.exp(-0.5 * np.sum(delta*sigma_inv_delta, axis=1))
    fx = norm*exponent
    
    # Jacobian
    jac = -sigma_inv_delta * fx
    
    # Hessian
    hess = sigma_inv_delta[:,:,np.newaxis]*sigma_inv_delta[:,np.newaxis,:] - sigma_inv
    hess = hess * fx[:,np.newaxis,np.newaxis]
    
    return -np.sum(fx*weight, axis=0), -np.sum(jac*weight[:,np.newaxis], axis=0), \
            -np.sum(hess*weight[:,np.newaxis,np.newaxis], axis=0)

In [46]:
def rootfinder_hess(X, sigma_inv, sigma_det, mu, weight, norm):
    
    delta = mu-X[:,np.newaxis]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = np.exp(-0.5 * np.sum(delta*sigma_inv_delta, axis=1))
    fx = norm*exponent
    
    # Hessian
    hess = sigma_inv_delta[:,:,np.newaxis]*sigma_inv_delta[:,np.newaxis,:] - sigma_inv
    hess = hess * fx[:,np.newaxis,np.newaxis]
    
    return -np.sum(hess*weight[:,np.newaxis,np.newaxis], axis=0)

In [104]:
def rootfinder_jac(X, sigma_inv, sigma_det, mu, weight, norm):
    
    delta = mu-X[np.newaxis,:]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = np.exp(-0.5 * np.sum(delta*sigma_inv_delta, axis=1))
    fx = norm*exponent
    
    # Jacobian
    jac = -sigma_inv_delta * fx[:,np.newaxis]
    
    # Hessian
    #hess = sigma_inv_delta[:,:,np.newaxis]*sigma_inv_delta[:,np.newaxis,:] - sigma_inv
    #hess = hess * fx[:,np.newaxis,np.newaxis]
    
    return 5-np.sum(fx*weight, axis=0), -np.sum(jac*weight[:,np.newaxis], axis=0)

In [102]:
def log_rootfinder(X, sigma_inv, sigma_det, mu, weight, lognorm):
    
    delta = mu-X[:,np.newaxis]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = -0.5 * np.sum(delta*sigma_inv_delta, axis=1)
    fx = lognorm + exponent
    
    # Jacobian
    jac = -sigma_inv_delta
    
    # Hessian
    hess = - sigma_inv
    
    return -np.sum(fx*weight, axis=0), -np.sum(jac*weight[:,np.newaxis], axis=0), \
            -np.sum(hess*weight[:,np.newaxis,np.newaxis], axis=0)

In [105]:
# Covariance matrix
sigma = np.zeros((params.shape[0], 2, 2))
sigma[:,[0,1],[0,1]] = params[:,[2,3]]**2
sigma[:,[0,1],[1,0]] = (params[:,2]*params[:,3]*params[:,4])[:,np.newaxis]

# Gaussian value
sigma_inv, sigma_det = quick_invdet(sigma)

norm = 1/(2*np.pi*np.sqrt(sigma_det))

In [87]:
%time rootfinder_jac(np.array([1.0, 1.0]), sigma_inv, sigma_det, params[:,:2], params[:,5], norm)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 159 µs


(-0.561286995909023, array([-0.06085081, -0.06085081]))

In [43]:
1/(2*np.pi*np.sqrt(sigma_det))

array([15.99567363,  0.03978874])

In [108]:
%time scipy.optimize.minimize(rootfinder_jac, np.array([0.9, 0.9]), method='Nelder-Mead', args=(sigma_inv, sigma_det, params[:,:2], params[:,5], norm), jac=True)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.5 ms


/home/andy/py3/lib/python3.5/site-packages/scipy/optimize/_minimize.py:501: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)


 final_simplex: (array([[0.33242197, 0.33243736],
       [0.33233345, 0.33242474],
       [0.33245224, 0.33235441]]), array([4.39707722, 4.39707722, 4.39707722]))
           fun: 4.397077216103094
       message: 'Optimization terminated successfully.'
          nfev: 60
           nit: 33
        status: 0
       success: True
             x: array([0.33242197, 0.33243736])

In [64]:
out = np.zeros(params.shape[0])
loc = np.zeros((params.shape[0], 2))
for i in range(params.shape[0]):
    opt = scipy.optimize.minimize(rootfinder_jac, params[i,:2], 
                                  args=(sigma_inv, sigma_det, params[:,:2], params[:,5], norm), jac=True)
    
    out[i] = opt.fun
    loc[i] = opt.x

In [65]:
out, loc

(array([-160.02992358,   -0.19894368]), array([[ 1.,  1.],
        [-1., -1.]]))

In [139]:
def jac_rootfinder(X, sigma_inv, sigma_det, mu, weight, lognorm):
    
    delta = mu-X[np.newaxis,:]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = np.exp(-0.5 * np.sum(delta*sigma_inv_delta, axis=1))
    fx = norm*exponent
    
    # Jacobian
    jac = -sigma_inv_delta * fx[:,np.newaxis]
    
    result =  -np.sum(jac*weight[:,np.newaxis], axis=0)
    #print(X.shape, result.shape)
    return result

In [141]:
def jac_rootfinder_hess(X, sigma_inv, sigma_det, mu, weight, lognorm):
    
    delta = mu-X[np.newaxis,:]
    sigma_inv_delta = np.sum(sigma_inv*delta[:,:,np.newaxis], axis=1)
    
    # Gaussian
    exponent = np.exp(-0.5 * np.sum(delta*sigma_inv_delta, axis=1))
    fx = norm*exponent
    
    # Jacobian
    jac = -sigma_inv_delta * fx[:,np.newaxis]
    
    # Hessian
    hess = sigma_inv_delta[:,:,np.newaxis]*sigma_inv_delta[:,np.newaxis,:] - sigma_inv
    hess = hess * fx[:,np.newaxis,np.newaxis]
    
    return -np.sum(jac*weight[:,np.newaxis], axis=0), \
            -np.sum(hess*weight[:,np.newaxis,np.newaxis], axis=0)

In [143]:
%time scipy.optimize.root(jac_rootfinder, np.array([-1., -1.]), args=(sigma_inv, sigma_det, params[:,:2], params[:,5], norm))#, jac=True)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 546 µs


    fjac: array([[-0.99377224,  0.1114304 ],
       [-0.1114304 , -0.99377224]])
     fun: array([-3.46944695e-18, -3.46944695e-18])
 message: 'The solution converged.'
    nfev: 9
     qtf: array([-1.84704874e-11, -2.31357399e-11])
       r: array([ 0.10754643, -0.02381862,  0.10487568])
  status: 1
 success: True
       x: array([0.33239067, 0.33239067])

In [145]:
%time scipy.optimize.root(jac_rootfinder_hess, np.array([-1., -1.]), args=(sigma_inv, sigma_det, params[:,:2], params[:,5], norm), jac=True)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 29.7 ms


    fjac: array([[-0.70710678,  0.70710678],
       [-0.70710678, -0.70710678]])
     fun: array([5.71593506e-91, 5.71593506e-91])
 message: 'The number of calls to function has reached maxfev = 300.'
    nfev: 300
    njev: 3
     qtf: array([ 2.62075589e-101, -1.61807204e-090])
       r: array([-3.29489841e-69,  3.29489841e-69, -3.35290799e-89])
  status: 2
 success: False
       x: array([-29.85507037, -29.85507037])

In [151]:
%%time

out = np.zeros(params.shape[0])
loc = np.zeros((params.shape[0], 2))
for i in range(params.shape[0]):
    opt = scipy.optimize.root(jac_rootfinder, params[i,:2], 
                                  args=(sigma_inv, sigma_det, params[:,:2], params[:,5], norm))
    loc[i] = opt.x

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.38 ms


In [152]:
loc

array([[0.33239067, 0.33239067],
       [0.33239067, 0.33239067],
       [0.33239067, 0.33239067]])